In [177]:
#import libraries needed
import numpy as np
import matplotlib.pyplot as plt

# Optimization via Gradient Descent

In this homework, we want to study methods to solve the general optimization problem where,
given a function $f : \mathbb{R}^n → \mathbb{R}$, we want to compute

$$ x^* = \underset{x ∈\mathbb{R}^n}{arg\,min}\;f(x) $$ 

In particular, we will consider the situation where $f(x)$ is at least differentiable, which implies that we can compute its gradient $∇f(x)$.

In this framework, one of the most common way to approach is to use the Gradient Descent (GD)
method, which is an iterative algorithm that, given an initial iterate $x_0 ∈\mathbb{R}^n$ and a positive parameter called
step size $α_k > 0$ for each iteration, computes

$$ x_{k+1} = x_k − α_k∇f(x_k) $$

You are asked to implement the GD method in Python and to test it with some remarkable functions.


- Write a script that implement the GD algorithm, with the following structure:

        Input:

                f: the function f(x) we want to optimize.

                It is supposed to be a Python function, not an array.

                grad_f: the gradient of f(x). It is supposed to be a Python function, not an array.

                x0: an n-dimensional array which represents the initial iterate.

                kmax: an integer. The maximum possible number of iterations (to avoid infinite loops)

                tolf: small float. The relative tollerance of the algorithm.

                Convergence happens if ||grad_f(x_k)||_2 < tolf ||grad_f(x_0)||_2
                tolx: small float. The tollerance in the input domain.
                Convergence happens if ||x_{k} - x_{k-1}||_2 < tolx.

        Pay attention to to the first iterate.
        

        Output:

                x: an array that contains the value of x_k FOR EACH iterate x_k (not only the latter).

                k: an integer. The number of iteration needed to converge. k < kmax.

                f_val: an array that contains the value of f(x_k) FOR EACH iterate x_k.

                grads: an array that contains the value of grad_f(x_k) FOR EACH iterate x_k.

                err: an array the contains the value of ||grad_f(x_k)||_2 FOR EACH iterate x_k.

        For the moment, consider a fixed value of α > 0.

<br>

- It's given the implementation of the backtracking algorithm for the GD method. That function works as follows:

        Input:
        
                f: the function f(x) we want to optimize.

                It is supposed to be a Python function, not an array.

                grad_f: the gradient of f(x). It is supposed to be a Python function, not an array.

                x: an array. The actual iterate x_k for which you want to find the correct value for alpha.

        Output:

                alpha: a float. The correct step size for the next iteration.

        Modify the code for the GD method to let it be able to use the backtracking algorithm for the choice of the step size

In [178]:
def f(x):
    return x**2 + 2*x +1

def grad_f(x):
    return (2*x + 2)

In [179]:
def backtracking(f, grad_f, x):
    """
    This function is a simple implementation of the backtracking algorithm for
    the GD (Gradient Descent) method.
    
    f: function. The function that we want to optimize.
    grad_f: function. The gradient of f(x).
    x: ndarray. The actual iterate x_k.
    """
    alpha = 1
    c = 0.8
    tau = 0.25
    
    while f(x - alpha * grad_f(x)) > f(x) - c * alpha * np.linalg.norm(grad_f(x)) ** 2:
        alpha = tau * alpha
        
        if alpha < 1e-3:
            break
    return alpha

In [180]:
def GD(f, grad_f, x0, kmax, tolf, tolx, step=0.001, useBackTracking=True):
    
    # Initialization
    k = 0

    dim_m, dim_n = (kmax+1, ) + x0.shape

    x = np.empty((dim_m, dim_n))
    f_val = np.empty((kmax+1, ))
    grads = np.empty((dim_m, dim_n))
    err_val = np.empty((kmax+1, ))
    
    # Assign the values for the first iteration
    x[k]=x0
    f_val[k] = f(x0)
    grads[k] = grad_f(x0)
    err_val[k] = np.linalg.norm(grad_f(x0))
    
    # Choose step size
    alpha = step
    
    # Handle the condition for the first iteration
    k+=1
    x[k]=x[k-1]-alpha*grad_f(x[k-1])
    f_val[k] = f(x[k])
    grads[k] = grad_f(x[k])
    err_val[k] = np.linalg.norm(grad_f(x[k]))

    #Conditions
    cond1 = (np.linalg.norm(grad_f(x[k])) > tolf * grad_f(x[k-1])).all()
    cond2 = (np.linalg.norm(x[k] - x[k-1]) > tolx * np.linalg.norm(x[k-1])).all()

    conditions = cond1 and cond2

    # Start the iterations
    while (conditions and k < kmax):

        #Update k
        k = k+1
        
        # Update the value of x
        x[k] = x[k-1]-alpha*grad_f(x[k-1])
        
        #Update alpha
        if useBackTracking:
            alpha = backtracking(f, grad_f, x[k])
        
        #Update values 
        f_val[k] = f(x[k])
        grads[k] = grad_f(x[k])
        err_val[k] = np.linalg.norm(grad_f(x[k]))

        #Update condintions
        cond1 = np.linalg.norm(grad_f(x[k])) > tolf * grad_f(x[k-1]).all()
        cond2 = np.linalg.norm(x[k] - x[k-1]) > tolx * np.linalg.norm(x[k-1]).all()
        conditions = cond1 and cond2
    
    #Truncate the vectors that are (eventually) too long
    x = x[:k+1]
    f_val = f_val[:k+1]
    grads = grads[:k+1]
    err_val = err_val[:k+1]
    
    return x, k, f_val, grads, err_val

In [181]:
x, k, fval, grads, errval = GD(f, grad_f, np.asarray([2]), 1000, 1e-6, 1e-6)
print(x[k])

[-0.9999938]


- Test the algorithm above on the following functions:
1. $f : \mathbb{R}^2 → \mathbb{R}$ such that

    $f(x_1, x_2) = (x_1 − 3)^2 + (x_2 − 1)^2$
    
    for which the true optimum is $x^* = (3, 1)^T$

<br>

2. $f : \mathbb{R}^2 → \mathbb{R}$ such that

    $f(x1, x2) = 10(x_1 − 1)^2 + (x_2 − 2)^2$

    for which the true optimum is $x^* = (1, 2)^T$

<br>

3. $f : \mathbb{R}^n → \mathbb{R}$ such that

    $f(x) = \frac{1}{2}||Ax − b||_2^2$

    where $A ∈ \mathbb{R}^{n×n}$ is the Vandermonde matrix associated with the vector $v ∈ \mathbb{R}^n$ that contains $n$ equispaced values in the interval $[0, 1]$, and $b ∈ \mathbb{R}^n$ is computed by first setting $x_{true} = (1, 1, . . . , 1)^T$ and then $b = Ax_{true}$. Try for different values of $n$ (e.g. $n = 5, 10, 15, . . .$)

<br>

4. $f : \mathbb{R}^n → \mathbb{R}$ such that

    $f(x) = \frac{1}{2}||Ax − b||_2^2+\frac{λ}{2}||x||_2^2$
    
    where $A$ and $b$ are the same of the exercise above, while $λ$ is a fixed value in the interval $[0, 1]$.
    Try different values for $λ$.

<br>

5. $f : \mathbb{R} -> \mathbb{R}$ suche that

    $f(x) = x^4 + x^3 -2x^2 - 2x$



- For each of the functions above, run the GD method with and without the backtracking, trying different values for the step size $\alpha > 0$ when you are not using backtracking. Observe the different behavior of GD.
<br>
- To help visualization, it is convenient to plot the error vector that contains the $||∇f(x_k)||_2$, to check that it goes to zero. Compare the convergence speed (in terms of the number of iterations k) in the different cases.
<br>
- For each of the points above, fix $x_0= (0, 0, ..., 0)^T$, kmax = 100, while choose your values for tolf and tolx. It is recommended to also plot the error $||x_k - x^*||_2$ varying k when the true $x^*$ is available.
- Only for the non-convex function defined in 5, plot it in the interval [-3; 3] and test the convergence point of GD with different values of $x_0$ and different step-sizes. Observe when the convergence point is the global minimum and when it stops on a local minimum or maximum.

In [182]:
eps = sys.float_info.epsilon

def f1(x):
    return (x[0]-3)**2 + (x[1]-1)**2

def grad_f1(x):
    grad = np.empty(len(x))
    grad[0] = 2*(x[0]-3)
    grad[1] = 2*(x[1]-1)

    return grad


def f2(x):
    return 10*(x[0]-1)**2 + (x[1]-2)**2    

def grad_f2(x):
    grad = np.empty(len(x))
    grad[0] = 20*(x[0]-1)
    grad[1] = 2*(x[1]-2)

    return grad


def f3(x, n):
    v = np.arange(0, 1+eps, 1/(n-1))
    A = np.vander(v)
    x_true = np.ones((n, ))
    b = A@x_true

    y = 1/2 * (np.linalg.norm(A@x - b))**2

    return y

def grad_f3(x, n):
    v = np.arange(0, 1+eps, 1/(n-1))
    A = np.vander(v)
    x_true = np.ones((n, ))
    b = A@x_true

    grad = A.T @ (A@x - b)
    
    return grad

def f4(x, n, L):
    v = np.arange(0, 1+eps, 1/(n-1))
    A = np.vander(v)
    x_true = np.ones((n, ))
    b = A@x_true

    y1 = 1/2 * (np.linalg.norm(A@x - b))**2
    y2 = L/2 * (np.linalg.norm(x))**2

    return y1 + y2

def grad_f4(x, n, L):
    v = np.arange(0, 1+eps, 1/(n-1))
    A = np.vander(v)
    x_true = np.ones((n, ))
    b = A@x_true

    grad = A.T @ (A@x - b) + L*x

    return grad

def f5(x):
    return x**4 + x**3 - 2*x**2 - 2*x

def grad_f5(x):
    return 4*x**3 + 3*x**2 - 4*x -2


---
# Test funciton 1

$f(x_1, x_2) = (x_1 − 3)^2 + (x_2 − 1)^2$

We are going to test the the GD methon on the function with and without backtracking.
When we are not using backtracking we are testing with learning rate of $1$, $0.5$, $0.1$, $0.03$ and $0.01$

In [183]:
x0 = np.asarray([0,0])
tolx = 1e-6
tolf = 1e-6
kmax = 10000

cvector = np.asarray([1, 0.5, 0.1, 0.03, 0.01])

f_tot = []
x_tot = []

for c in cvector:
    x, k, f_val, grads, err_val  = GD(f1, grad_f1, x0, kmax, tolf, tolx, step=c, useBackTracking=False)
    plt.plot()
    
    f_tot.append(f_val)
    x_tot.append(x)